# Interactive SG
## A no-code tool to create your own sinusoidal grating dataset

[Click to see advanced user (PyTorch-compatible) version](https://hydo.ai)

In [1]:
!pip install ipyfilechooser ipywidgets scipy numpy matplotlib -qqq

In [2]:
from scipy import ndimage
import numpy as np
from PIL import Image, ImageDraw, ImageFilter
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from ipyfilechooser import FileChooser
import pickle

In [3]:
from SineGratesDataset import SineGrates, pil_to_tensor, circular_sinegrate, show_img

Sinusoid $ I(\mathbf{x}) = A \cos(\mathbf{\omega} \mathbf{x} + \rho) $

where
+ $ A $ is the amplitude
+ $ \rho $ is the phase
+ $ \mathbf{\omega} $ is the frequency.

Here's an interactive demonstration of the parameters that can be adjusted to create different kinds of sinusoidal gratings. Try adjusting the sliders!

In [4]:
def demo_sinusoidal_grating(frequency, rotation, phase_shift):
    show_img(circular_sinegrate(frequency, rotation, phase_shift=phase_shift, image_size=(256,256)))
interact(demo_sinusoidal_grating, frequency=(1, 20, 0.1), rotation=(0, 180, 1), phase_shift=(0, 2*np.pi, 0.1))

interactive(children=(FloatSlider(value=10.0, description='frequency', max=20.0, min=1.0), IntSlider(value=90,…

<function __main__.demo_sinusoidal_grating(frequency, rotation, phase_shift)>

# Defining the distribution parameters for a dataset of sinusoidal gratings

In [5]:
scheme_w = widgets.ToggleButtons(
    options=['Rule-Based', 'Information-Integration'],
    description='Scheme:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    #tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

In [6]:
display(scheme_w)

ToggleButtons(description='Scheme:', options=('Rule-Based', 'Information-Integration'), value='Rule-Based')

In [7]:
int_w = widgets.IntText(
    value=100,
    description='Dataset size:',
    disabled=False
)

In [8]:
display(int_w)

IntText(value=100, description='Dataset size:')

In [9]:
bool_w = widgets.Checkbox(
    value=False,
    description='Randomize phase',
    disabled=False,
    indent=False
)

In [10]:
display(bool_w)

Checkbox(value=False, description='Randomize phase', indent=False)

In [11]:
from IPython.display import display
button = widgets.Button(description="See/modify distribution")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    if scheme_w.value == 'Rule-Based':
        cat_scheme = 'rb'
    elif scheme_w.value == 'Information-Integration':
        cat_scheme = 'ii'
    dataset_length = int_w.value // 2 # we need to split into 2, because condition 'a' and 'b' datasets are generated separately
    randomize_phase = bool_w.value
    rb_params = {
            'a_means':[[30,50],[50,70]],
            'b_means':[[50,30],[70,50]],
            'a_covariances':[[[10,0],[0,150]],[[150,0],[0,10]]],
            'b_covariances':[[[10,0],[0,150]],[[150,0],[0,10]]]
        }

    ii_params = {
        'a_means':[40,50],
        'b_means':[60,50],
        'a_covariances':[[10,0],[0,280]],
        'b_covariances':[[10,0],[0,280]]
    }
    
    
    if cat_scheme == 'rb':
        dist_params = rb_params
    elif cat_scheme == 'ii':
        dist_params = ii_params
    dataset = SineGrates(cat_scheme=cat_scheme, dist_params=dist_params, length=dataset_length, transform=None, randomize_phase=randomize_phase)
    def modify_dist_params_ii(**kwargs):
        new_params = {
            'a_means':[kwargs['a_means_x'], kwargs['a_means_y']],
            'b_means':[kwargs['b_means_x'], kwargs['b_means_y']],
            'a_covariances':[[kwargs['a_cov_x'],0],[0,kwargs['a_cov_y']]],
            'b_covariances':[[kwargs['b_cov_x'],0],[0,kwargs['b_cov_y']]]
        }
        dataset.set_dist_params(new_params)
        pickle.dump(dataset, open('temp_distribution.pkl', 'wb'))
        plt.show(dataset.plot_final())
    def modify_dist_params_rb(**kwargs):
        new_params = {
            'a_means':[[kwargs['a_means_1_x'], kwargs['a_means_1_y']],
                       [kwargs['a_means_2_x'], kwargs['a_means_2_y']]],
            'b_means':[[kwargs['b_means_1_x'], kwargs['b_means_1_y']],
                       [kwargs['b_means_2_x'], kwargs['b_means_2_y']]],
            'a_covariances':[[[kwargs['a_cov_1_x'],0],[0,kwargs['a_cov_1_y']]],
                             [[kwargs['a_cov_2_x'],0],[0,kwargs['a_cov_2_y']]]],
            'b_covariances':[[[kwargs['b_cov_1_x'],0],[0,kwargs['b_cov_1_y']]],
                             [[kwargs['b_cov_2_x'],0],[0,kwargs['b_cov_2_y']]]],
        }
        dataset.set_dist_params(new_params)
        pickle.dump(dataset, open('temp_distribution.pkl', 'wb'))
        plt.show(dataset.plot_final())
        
    with output:
        if cat_scheme == 'ii':
            interact(modify_dist_params_ii, 
                     a_means_x = (0,80,1),
                     a_means_y = (0,100,1),
                     b_means_x = (0,120,1),
                     b_means_y = (0,100,1),
                     a_cov_x = (0,20,1),
                     a_cov_y = (0,560, 1),
                     b_cov_x = (0,20,1),
                     b_cov_y = (0,560, 1),
                    )
        elif cat_scheme == 'rb':
                interact(modify_dist_params_rb, 
                         a_means_1_x = (0,60,1),
                         a_means_1_y = (0,100,1),
                         a_means_2_x = (0,100,1),
                         a_means_2_y = (0,140,1),
                         b_means_1_x = (0,100,1),
                         b_means_1_y = (0,60,1),
                         b_means_2_x = (0,140,1),
                         b_means_2_y = (0,100,1),
                         a_cov_1_x = (0,20,1),
                         a_cov_1_y = (0,300,1),
                         a_cov_2_x = (0,300,1),
                         a_cov_2_y = (0,20,1),
                         b_cov_1_x = (0,20,1),
                         b_cov_1_y = (0,300,1),
                         b_cov_2_x = (0,300,1),
                         b_cov_2_y = (0,20,1),
                    )
        print("")

button.on_click(on_button_clicked)



Button(description='See/modify distribution', style=ButtonStyle())

Output()

Using this folder selector, choose where to save the images:

In [12]:
fc = FileChooser('.')
display(fc)

FileChooser(path='/home/jasonsohn/Repos/deep-sinusoidal-grating', filename='', title='', show_hidden=False, se…

Using the distribution above, we actually generate the sinusoidal grating images:

In [13]:
#from IPython.display import display
button_2 = widgets.Button(description="Save dataset")
output_2 = widgets.Output()

display(button_2, output_2)

def on_button_clicked_2(b):

    with output_2:
        print("Generating dataset...")
        dataset = pickle.load(open('temp_distribution.pkl', 'rb'))
        dataset.save_dataset(fc.selected_path)
        print(f"{len(dataset)} examples have been created and saved to {fc.selected_path}.")

button_2.on_click(on_button_clicked_2)

Button(description='Save dataset', style=ButtonStyle())

Output()